In [1]:
import pandas as pd

# Load the datasets
file_path_internet = '/content/API_IT.NET.USER.ZS_DS2_en_csv_v2_213153.csv'
file_path_poverty = '/content/API_SI.POV.DDAY_DS2_en_csv_v2_227712.csv'
file_path_unemployment = '/content/API_SL.UEM.TOTL.ZS_DS2_en_csv_v2_228704.csv'

# Read the files, skipping the initial rows that are not data headers
data_internet = pd.read_csv(file_path_internet, skiprows=4)
data_poverty = pd.read_csv(file_path_poverty, skiprows=4)
data_unemployment = pd.read_csv(file_path_unemployment, skiprows=4)

# Filter for Brazil
brazil_internet = data_internet[data_internet['Country Name'] == 'Brazil']
brazil_poverty = data_poverty[data_poverty['Country Name'] == 'Brazil']
brazil_unemployment = data_unemployment[data_unemployment['Country Name'] == 'Brazil']

# Define common columns for merging
# Define common columns for merging
common_columns = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'] + \
    [str(year) for year in range(1960, 2024)]  # Including years 1960 to 2023

# Concatenate data vertically, aligning by the specified common columns
brazil_combined = pd.concat([
    brazil_internet[common_columns],
    brazil_poverty[common_columns],
    brazil_unemployment[common_columns]
], axis=0)

# Reset index to make it clean
brazil_combined.reset_index(drop=True, inplace=True)



In [2]:
brazil_combined

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Brazil,BRA,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,54.551002,58.327952,60.87254,67.471285,70.434283,73.91244,81.342694,80.689893,80.527751,NaN
1,Brazil,BRA,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,3.300000,3.900000,4.70000,5.300000,5.300000,5.40000,2.000000,5.800000,3.500000,NaN
2,Brazil,BRA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.760000,8.540000,11.58000,12.790000,12.330000,11.94000,13.700000,13.160000,9.230000,7.776


In [3]:
# Paths to the metadata files
metadata_file_internet = '/content/Metadata_Indicator_API_IT.NET.USER.ZS_DS2_en_csv_v2_213153.csv'
metadata_file_poverty = '/content/Metadata_Indicator_API_SI.POV.DDAY_DS2_en_csv_v2_227712.csv'
metadata_file_unemployment = '/content/Metadata_Indicator_API_SL.UEM.TOTL.ZS_DS2_en_csv_v2_228704.csv'

# Load the metadata files
metadata_internet = pd.read_csv(metadata_file_internet)
metadata_poverty = pd.read_csv(metadata_file_poverty)
metadata_unemployment = pd.read_csv(metadata_file_unemployment)

# Select only the relevant columns from each metadata DataFrame
columns_to_keep = ['INDICATOR_CODE', 'SOURCE_NOTE', 'SOURCE_ORGANIZATION']
metadata_internet = metadata_internet[columns_to_keep]
metadata_poverty = metadata_poverty[columns_to_keep]
metadata_unemployment = metadata_unemployment[columns_to_keep]

# Create a single metadata DataFrame by concatenating all metadata DataFrames
metadata_combined = pd.concat([metadata_internet, metadata_poverty, metadata_unemployment], ignore_index=True)

# Merge the combined metadata with the brazil_combined DataFrame on 'Indicator Code'
brazil_combined = brazil_combined.merge(metadata_combined, how='left', left_on='Indicator Code', right_on='INDICATOR_CODE')

# Drop the extra 'INDICATOR_CODE' column if it's redundant
brazil_combined.drop('INDICATOR_CODE', axis=1, inplace=True)



In [4]:
brazil_combined

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,SOURCE_NOTE,SOURCE_ORGANIZATION
0,Brazil,BRA,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,60.87254,67.471285,70.434283,73.91244,81.342694,80.689893,80.527751,NaN,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...
1,Brazil,BRA,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,4.70000,5.300000,5.300000,5.40000,2.000000,5.800000,3.500000,NaN,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D..."
2,Brazil,BRA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,11.58000,12.790000,12.330000,11.94000,13.700000,13.160000,9.230000,7.776,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...


In [5]:
metadata_country_path = '/content/Metadata_Country_API_IT.NET.USER.ZS_DS2_en_csv_v2_213153.csv'

In [6]:
metadata_country = pd.read_csv(metadata_country_path)

In [7]:
brazil_metadata = metadata_country[metadata_country['Country Code'] == 'BRA']

In [8]:
brazil_metadata

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
29,BRA,Latin America & Caribbean,Upper middle income,NaN,Brazil,NaN


In [9]:

brazil_region = brazil_metadata['Region'].values[0] if not brazil_metadata['Region'].empty else 'Not Available'
brazil_income_group = brazil_metadata['IncomeGroup'].values[0] if not brazil_metadata['IncomeGroup'].empty else 'Not Available'


In [10]:
brazil_combined['Region'] = brazil_region
brazil_combined['IncomeGroup'] = brazil_income_group

In [11]:
empty_columns = brazil_combined.columns[brazil_combined.isna().all()].tolist()

In [12]:
brazil_combined.dropna(axis=1, how='all', inplace=True)


In [13]:
print("Deleted columns:", empty_columns)

Deleted columns: ['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980']


In [14]:
brazil_combined

,Country Name,Country Code,Indicator Name,Indicator Code,1981,1982,1983,1984,1985,1986,...,2018,2019,2020,2021,2022,2023,SOURCE_NOTE,SOURCE_ORGANIZATION,Region,IncomeGroup
0,Brazil,BRA,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,70.434283,73.91244,81.342694,80.689893,80.527751,NaN,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income
1,Brazil,BRA,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,23.9,24.4,30.6,29.9,25.8,12.3,...,5.300000,5.40000,2.000000,5.800000,3.500000,NaN,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income
2,Brazil,BRA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,12.330000,11.94000,13.700000,13.160000,9.230000,7.776,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...,Latin America & Caribbean,Upper middle income


In [15]:
nan_count_before = brazil_combined.isna().sum().sum()

# Replace NaN values with "Data Not Available"
brazil_combined.fillna('Data Not Available', inplace=True)

# Print the number of NaN values replaced
print(f"Total NaN values replaced: {nan_count_before}")


Total NaN values replaced: 25


In [16]:
year_columns = [col for col in brazil_combined.columns if col.isdigit()]

# Specify the ID variables for the melt function
id_vars = [col for col in brazil_combined.columns if col not in year_columns]

# Melt the DataFrame to have years as rows
brazil_melted = pd.melt(brazil_combined, id_vars=id_vars, value_vars=year_columns,
                        var_name='Year', value_name='Value')

# Optionally, reorder the columns for better readability
brazil_melted = brazil_melted[['Year', 'Country Name', 'Indicator Name', 'Indicator Code',
                               'SOURCE_NOTE', 'SOURCE_ORGANIZATION', 'Region', 'IncomeGroup', 'Value']]



In [17]:
brazil_melted

,Year,Country Name,Indicator Name,Indicator Code,SOURCE_NOTE,SOURCE_ORGANIZATION,Region,IncomeGroup,Value
0,1981,Brazil,Individuals using the Internet (% of population),IT.NET.USER.ZS,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income,Data Not Available
1,1981,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,23.9
2,1981,Brazil,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...,Latin America & Caribbean,Upper middle income,Data Not Available
3,1982,Brazil,Individuals using the Internet (% of population),IT.NET.USER.ZS,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income,Data Not Available
4,1982,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,24.4
...,...,...,...,...,...,...,...,...,...
124,2022,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,3.5
125,2022,Brazil,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...,Latin America & Caribbean,Upper middle income,9.23
126,2023,Brazil,Individuals using the Internet (% of population),IT.NET.USER.ZS,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income,Data Not Available
127,2023,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,Data Not Available


In [18]:
import pandas as pd

# Assuming 'brazil_melted' is already loaded and contains the 'Value' column
# Filter out rows where the 'Value' column is "Data Not Available"
brazil_melted_filtered = brazil_melted[brazil_melted['Value'] != "Data Not Available"]

# Now, brazil_melted_filtered contains only the rows where data is available
# Optionally, you can save this cleaned DataFrame to a new CSV file
brazil_melted_filtered.to_csv('brazil_yearly_data_filtered.csv', index=False)

print("Filtered data saved to 'brazil_yearly_data_filtered.csv'")
print(f"Removed rows: {len(brazil_melted) - len(brazil_melted_filtered)}")


Filtered data saved to 'brazil_yearly_data_filtered.csv'
Removed rows: 25


In [22]:
brazil_melted_filtered

,Year,Country Name,Indicator Name,Indicator Code,SOURCE_NOTE,SOURCE_ORGANIZATION,Region,IncomeGroup,Value
1,1981,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,23.9
4,1982,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,24.4
7,1983,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,30.6
10,1984,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,29.9
13,1985,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,25.8
...,...,...,...,...,...,...,...,...,...
122,2021,Brazil,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...,Latin America & Caribbean,Upper middle income,13.16
123,2022,Brazil,Individuals using the Internet (% of population),IT.NET.USER.ZS,Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,Latin America & Caribbean,Upper middle income,80.527751
124,2022,Brazil,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,Poverty headcount ratio at $2.15 a day is the ...,"World Bank, Poverty and Inequality Platform. D...",Latin America & Caribbean,Upper middle income,3.5
125,2022,Brazil,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,Unemployment refers to the share of the labor ...,International Labour Organization. “ILO Modell...,Latin America & Caribbean,Upper middle income,9.23


In [20]:
import pandas as pd

# Assuming 'brazil_melted' is already loaded and transformed as required
# Define a function to determine the category based on the indicator name
def determine_category(indicator_name):
    if "Internet" in indicator_name:
        return "Internet users"
    elif "Unemployment" in indicator_name:
        return "Unemployment"
    elif "Poverty" in indicator_name:
        return "Poverty"
    else:
        return "Other"  # Default category if none of the keywords match

# Apply the function to create the new 'category' column
brazil_melted_filtered['category'] = brazil_melted_filtered['Indicator Name'].apply(determine_category)

# Optionally, check the distribution of categories to ensure the mapping is correct
print(brazil_melted_filtered['category'].value_counts())




category
Internet users    43
Poverty           43
Unemployment      43
Name: count, dtype: int64
